In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3



In [2]:
import gdown
url = "https://drive.google.com/file/d/18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl/view?usp=sharing"
file_id = url.split("/")[-2]
file_id

'18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl'

In [3]:
prefix = "https://drive.google.com/uc?export=download&id="
gdown.download(prefix+file_id, "chest_data.zip")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl
From (redirected): https://drive.google.com/uc?export=download&id=18uXV88Z3PjWkYib-dKHVMr55VrqiiFrl&confirm=t&uuid=8baf1e08-43b2-45c6-b49a-8feea48b5b55
To: /content/chest_data.zip
100%|██████████| 124M/124M [00:01<00:00, 98.4MB/s]


'chest_data.zip'

In [5]:
import zipfile
unzip_path = '/content'
os.makedirs(unzip_path, exist_ok=True)
with zipfile.ZipFile("chest_data.zip", 'r') as zip_ref:
  zip_ref.extractall(unzip_path)

In [7]:
# File Directory for both the train and test
train_path = "../content/Data/train"
val_path = "../content/Data/valid"
test_path = "../content/Data/test"

In [8]:
# Define function to count number of images per class using a dictionary
def GetDatasetSize(path):
    num_of_image = {}
    for folder in os.listdir(path):
        # count files in the folder
        num_of_image[folder] = len(os.listdir(os.path.join(path, folder)));
    return num_of_image;

# Get the number of images per class in each set (train, validation and test)
train_set = GetDatasetSize(train_path)
val_set = GetDatasetSize(val_path)
test_set = GetDatasetSize(test_path)
print(train_set,"\n\n",val_set,"\n\n",test_set)

{'normal': 148, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 115, 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 195, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 155} 

 {'normal': 13, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 21, 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 23, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 15} 

 {'normal': 54, 'squamous.cell.carcinoma': 90, 'large.cell.carcinoma': 51, 'adenocarcinoma': 120}


# DATA PREPROCESSING

In [9]:
# Image data generator with specified augmentation configurations (mostly geometric transformations)
train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest',
                                  zoom_range=0.2,
                                  shear_range = 0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  rotation_range=0.4)

# Using data generator to create augmented data from image files in train_path directory
train_data = train_datagen.flow_from_directory(train_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')

# Dicctionary with class names to their respective indices in the generated data
train_data.class_indices

Found 613 images belonging to 4 classes.


{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0,
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1,
 'normal': 2,
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}

In [10]:
val_datagen = ImageDataGenerator(rescale = 1.0/255.0)
val_data = val_datagen.flow_from_directory(val_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')
val_data.class_indices

Found 72 images belonging to 4 classes.


{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0,
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1,
 'normal': 2,
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}

In [11]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.0)
test_data = test_datagen.flow_from_directory(test_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')
test_data.class_indices

Found 315 images belonging to 4 classes.


{'adenocarcinoma': 0,
 'large.cell.carcinoma': 1,
 'normal': 2,
 'squamous.cell.carcinoma': 3}

# EfficientNetB0


In [12]:
base_model = EfficientNetB0(input_shape = (350, 350, 3),
                         include_top = False,
                         weights = 'imagenet')

16705208/16705208 [==============================] - 0s 0us/step


In [13]:
mc = ModelCheckpoint(
    filepath="./ct_effnet_best_model.hdf5",
    monitor= 'val_accuracy',
    verbose= 1,
    save_best_only= True,
    mode = 'auto'
    );

call_back = [ mc];

tensorboard = TensorBoard(log_dir = 'logs')
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

#We also add a earlystop for prevent the computer from wasting time if it's not making progress.


early_stopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

# Training the model

In [14]:
#Customize our base model
EffNetmodel = base_model.output
EffNetmodel = tf.keras.layers.GlobalAveragePooling2D()(EffNetmodel)
# to provide overfitting problem
EffNetmodel = tf.keras.layers.Dropout(rate=0.5)(EffNetmodel)

#Finally, we add a layer with 4 'neurons' that will help us classify things into different categories.
EffNetmodel = tf.keras.layers.Dense(4,activation='softmax')(EffNetmodel)

In [15]:
#We put together the original EfficientNetB0 and our new custom parts
EffNetmodel = tf.keras.models.Model(inputs=base_model.input, outputs = EffNetmodel)

#Compile up the rules and tools for training your neural network.
#The optimizer determines how the model should update its internal parameters, the loss function quantifies how well the model is doing, and the metrics provide additional measures to track the model's performance.
EffNetmodel.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [16]:
%time
EffNetB0 = EffNetmodel.fit(
    train_data,
    steps_per_epoch = train_data.samples//train_data.batch_size,
    epochs = 64,
    validation_data = val_data,
    validation_steps = val_data.samples//val_data.batch_size,
    callbacks = [tensorboard, mc, reduce_lr, early_stopping]
    )

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
Epoch 1/64
122/122 [==============================] - ETA: 0s - loss: 1.1276 - accuracy: 0.4934
Epoch 1: val_accuracy improved from -inf to 0.15714, saving model to ./ct_effnet_best_model.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - 78s 251ms/step - loss: 1.1276 - accuracy: 0.4934 - val_loss: 1.7207 - val_accuracy: 0.1571 - lr: 0.0010
Epoch 2/64
122/122 [==============================] - ETA: 0s - loss: 0.8659 - accuracy: 0.6365
Epoch 2: val_accuracy improved from 0.15714 to 0.22857, saving model to ./ct_effnet_best_model.hdf5


122/122 [==============================] - 30s 245ms/step - loss: 0.8659 - accuracy: 0.6365 - val_loss: 1.5016 - val_accuracy: 0.2286 - lr: 0.0010
Epoch 3/64
122/122 [==============================] - ETA: 0s - loss: 0.7668 - accuracy: 0.6826
Epoch 3: val_accuracy did not improve from 0.22857


122/122 [==============================] - 28s 231ms/step - loss: 0.7668 - accuracy: 0.6826 - val_loss: 1.5000 - val_accuracy: 0.1143 - lr: 0.0010
Epoch 4/64
122/122 [==============================] - ETA: 0s - loss: 0.6634 - accuracy: 0.7336
Epoch 4: val_accuracy improved from 0.22857 to 0.30000, saving model to ./ct_effnet_best_model.hdf5


...

122/122 [==============================] - 27s 219ms/step - loss: 0.0923 - accuracy: 0.9655 - val_loss: 0.2576 - val_accuracy: 0.9286 - lr: 9.4143e-16
Epoch 61/64
122/122 [==============================] - ETA: 0s - loss: 0.0896 - accuracy: 0.9753
Epoch 61: val_accuracy did not improve from 0.94286


122/122 [==============================] - 27s 219ms/step - loss: 0.0896 - accuracy: 0.9753 - val_loss: 0.2498 - val_accuracy: 0.9286 - lr: 2.8243e-16
Epoch 62/64
122/122 [==============================] - ETA: 0s - loss: 0.0955 - accuracy: 0.9753
Epoch 62: val_accuracy did not improve from 0.94286

Epoch 62: ReduceLROnPlateau reducing learning rate to 8.472885923856935e-17.


122/122 [==============================] - 27s 219ms/step - loss: 0.0955 - accuracy: 0.9753 - val_loss: 0.2554 - val_accuracy: 0.9286 - lr: 2.8243e-16
Epoch 63/64
122/122 [==============================] - ETA: 0s - loss: 0.0800 - accuracy: 0.9819
Epoch 63: val_accuracy did not improve from 0.94286


122/122 [==============================] - 27s 218ms/step - loss: 0.0800 - accuracy: 0.9819 - val_loss: 0.2007 - val_accuracy: 0.9429 - lr: 8.4729e-17
Epoch 64/64
122/122 [==============================] - ETA: 0s - loss: 0.0857 - accuracy: 0.9753
Epoch 64: val_accuracy did not improve from 0.94286

Epoch 64: ReduceLROnPlateau reducing learning rate to 2.541865737452411e-17.


122/122 [==============================] - 27s 220ms/step - loss: 0.0857 - accuracy: 0.9753 - val_loss: 0.2022 - val_accuracy: 0.9429 - lr: 8.4729e-17


# Model Evaluation

In [18]:
# Loading the Best Fit Model
model_eff = load_model("./ct_effnet_best_model.hdf5")
# Checking the Accuracy of the Model
accuracy_effnet = model_eff.evaluate_generator(generator= test_data)[1]
loss_effnet = model_eff.evaluate_generator(generator= test_data)[0]
print(f"The accuracy of the model is = {accuracy_effnet*100} %")
print(f"The loss of the model is = {loss_effnet} %")

<ipython-input-18-8fd514e21431>:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  accuracy_effnet = model_eff.evaluate_generator(generator= test_data)[1]
<ipython-input-18-8fd514e21431>:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_effnet = model_eff.evaluate_generator(generator= test_data)[0]


The accuracy of the model is = 94.60317492485046 %
The loss of the model is = 0.24305105209350586 %
